# Get a list of Targets based on a pathway
#### Create a POST request to the RaMP API

In [101]:
import requests
import pandas as pd
import json

ramp_url = "https://rampdb.nih.gov/api/analytes-from-pathways"

post_body = {
    "analyte_type": "gene",
    "names_or_ids": "names",
    "max_pathway_size": 500,
    "pathway": ["Transcriptional Regulation by VENTX"]
}

#### do the POST request

In [102]:
response = requests.post(ramp_url, data=post_body)
response_json = json.loads(response.text)["data"]

#### Parse the result

In [103]:
target_uniprots = [id.split(":")[1].split(",")[0] 
                   for entry in response_json 
                   for id in entry["sourceAnalyteIDs"].split(";") if id.strip().startswith("uniprot:")]

target_uniprots = list(set(target_uniprots))

print(pd.DataFrame(target_uniprots))

         0
0   Q9UL18
1   P60006
2   Q9UJX6
3   O95231
4   Q9UJU2
5   Q8NDV7
6   Q9UJX3
7   P42771
8   Q96DE5
9   Q8NHZ8
10  Q9HCJ0
11  Q9H9G7
12  P30260
13  Q9NYG5
14  Q9UJX2
15  Q9UJX5
16  Q9H1A4
17  P04637
18  P17676
19  Q9HCK5
20  Q04206
21  Q9UM11
22  Q13042
23  Q9UM13
24  Q9UPQ9
25  Q9NQB0
26  Q9UJX4


# Get gene-disease associations and drug activities for those targets
#### Create a POST request to the Pharos API

In [104]:
pharos_url = "https://pharos-api.ncats.io/graphql"

post_body = {
  "query": """query targetDetails($targets: [String]){
  targets(targets: $targets) {
    count
    targets(top: 100000) {
      name
      uniprot
      ligands(isdrug: true) {
        name
        smiles
        synonyms {
          value
          name
        }
      }
      diseases {
        mondoID
        name
        mondoEquivalents {
          id
          name
        }
      }
    }
  }
}""",
  "variables": {"targets": target_uniprots},
  "operationName": "targetDetails"
}

#### do the POST request

In [105]:
response = requests.post(pharos_url, json=post_body)
response_json = json.loads(response.text)["data"]
target_list = response_json["targets"]["targets"]

#### Parse the disease results

In [106]:
disease_objects = [disease for target in target_list for disease in target["diseases"]]
disease_names = list(set(disease["name"] for disease in disease_objects))

print(pd.DataFrame(disease_objects))

gard_aliases = [alias["id"] for disease in disease_objects if disease["mondoEquivalents"] is not None for alias in disease["mondoEquivalents"] if alias["id"].startswith("GARD:")]
gard_aliases = list(set(gard_aliases))

           mondoID                                               name  \
0    MONDO:0005575                                  colorectal cancer   
1    MONDO:0004381  pancreatic intraductal papillary-mucinous neop...   
2    MONDO:0009807                                       osteosarcoma   
3    MONDO:0008170                                     ovarian cancer   
4    MONDO:0018177                                       glioblastoma   
..             ...                                                ...   
195  MONDO:0004948                B-cell chronic lymphocytic leukemia   
196  MONDO:0007576                                  esophageal cancer   
197  MONDO:0006639                           adrenal cortex carcinoma   
198  MONDO:0008734               adrenocortical carcinoma, hereditary   
199  MONDO:0100342                                   malignant glioma   

                                      mondoEquivalents  
0    [{'id': 'DOID:5672', 'name': 'large intestine ...  
1    [{'i

#### Parse the drug results

In [107]:
drug_objects = [drug for target in target_list for drug in target["ligands"]]
drug_names = list(set(drug["name"] for drug in drug_objects))

print(pd.DataFrame(drug_objects))

unii_aliases = [synonym["value"] for drug in drug_objects for synonym in drug["synonyms"] if synonym["name"] == "unii"]
unii_aliases = list(set(unii_aliases))


         name                                             smiles  \
0  bortezomib  CC(C)C[C@H](NC(=O)[C@H](Cc1ccccc1)NC(=O)c1cncc...   
1   gefitinib     COc1cc2ncnc(c2cc1OCCCN1CCOCC1)Nc1ccc(c(c1)Cl)F   

                                            synonyms  
0  [{'value': '69G8BD63PP', 'name': 'unii'}, {'va...  
1  [{'value': 'S65743JHBS', 'name': 'unii'}, {'va...  


## Get Disease Details
#### Create POST request to the RDAS API

In [108]:
rdas_url = "https://rdas.ncats.nih.gov/api/diseases"

post_body = {
    "query": """query diseaseQuery($where: GARDWhere) {
  gards(where: $where) {
    GardId
    GardName
    MeSH
    Synonyms
    COUNT_ARTICLES
    COUNT_TRIALS
    ClassificationLevel
  }
}""", 

  "variables": {"where": {"GardId_IN": gard_aliases}},
  "operationName": "diseaseQuery"
}

#### do the POST request

In [109]:
response = requests.post(rdas_url, json=post_body)
response_json = json.loads(response.text)["data"]

#### parse the results

In [110]:
gard_data = [{
    "GardId": obj["GardId"],
    "GardName": obj["GardName"],
    "COUNT_ARTICLES": obj["COUNT_ARTICLES"],
    "COUNT_TRIALS": obj["COUNT_TRIALS"],
    "ClassificationLevel": obj["ClassificationLevel"]
} for obj in response_json["gards"]]
df = pd.DataFrame(gard_data)
print(df)

          GardId                                       GardName  \
0   GARD:0005786                  Amyotrophic lateral sclerosis   
1   GARD:0006383                         Carcinoma of esophagus   
2   GARD:0009953                              Oligodendroglioma   
3   GARD:0004214                    Papilloma of choroid plexus   
4   GARD:0007392  Behavioral variant of frontotemporal dementia   
5   GARD:0006902                           Li-Fraumeni syndrome   
6   GARD:0002491                                   Glioblastoma   
7   GARD:0006996                      Cartilage-hair hypoplasia   
8   GARD:0009371                            Pituitary carcinoma   
9   GARD:0007026                           Pleural mesothelioma   
10  GARD:0007471                 Progressive supranuclear palsy   
11  GARD:0006225             Rare cutaneous lupus erythematosus   
12  GARD:0012757                         Acute myeloid leukemia   
13  GARD:0006353                                     Ependymom

## Get Drug Details
#### Create GET request strings

In [111]:
drug_requests = [f"https://drugs.ncats.io/api/v1/substances({unii})?view=full" for unii in unii_aliases]

drug_objects = []
for req in drug_requests:
    
    drug_json = json.loads(requests.get(req).text)
    
    if "_name" in drug_json:
        drug_obj = {
            "name": drug_json["_name"], 
            "approval_ID": drug_json["approvalID"],
            "substanceClass": drug_json["substanceClass"],
            "status": drug_json["status"]
        }
        if "structure" in drug_json:
            drug_obj["formula"] = drug_json["structure"]["formula"]
            drug_obj["mwt"] = drug_json["structure"]["mwt"]
        
        drug_objects.append(drug_obj)
    else:
        print(drug_json)

df = pd.DataFrame(drug_objects)
print(df)

         name approval_ID substanceClass    status        formula      mwt
0  BORTEZOMIB  69G8BD63PP       chemical  approved    C19H25BN4O4  384.237
1   GEFITINIB  S65743JHBS       chemical  approved  C22H24ClFN4O3  446.902
